In [1]:
import os
import sys
import io
import json
import contextlib
from pathlib import Path
root = Path().resolve()
while root != root.parent and not (root / 'backend').exists():
    root = root.parent
backend_root = root / 'backend'
if str(backend_root) not in sys.path:
    sys.path.insert(0, str(backend_root))
# 로그 최소화
os.environ['RAG_LOG_RETRIEVER_DEBUG'] = '0'
os.environ['RAG_LOG_TIMING'] = '0'
from app.rag.tests.test_suite import GUIDE_TESTS
from app.rag.pipeline import RAGConfig, run_rag


In [2]:
results = []
for t in GUIDE_TESTS:
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        res = await run_rag(t['query'], config=RAGConfig(top_k=4, enable_consult_search=True))
    log = buf.getvalue().strip()
    results.append({"guidanceScript": res.get('guidanceScript', ''), "log": log})
print(json.dumps(results, ensure_ascii=False, indent=2))


[
  {
    "guidanceScript": "고객님께서 나라사랑카드를 분실하셨습니다.\n나라사랑카드 분실 시 즉시 해당 은행(카드사)으로 신고하여 금융 사고를 예방해 주셔야 합니다.\n분실 신고은 안내드린 2주 기준으로 진행해도 괜찮을까요?",
    "log": "[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=109.5 rows=0\n[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=27.6 rows=5\n[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=135.8 rows=1\n[retriever] source=guide_all fetch_ms=0.0 cand_added=6 total_cand=6 break_hit=False\n[pipeline_retrieve] retrieve_ms=292.5 doc_count=4 route=card_usage\n[card_cache] set layer=redis key=7aa0333fcd6944a5 ttl=120\n[card_cache] set layer=mem key=7aa0333fcd6944a5 ttl=120\n[guide_docs] selected=[{'id': 'narasarang_faq_005', 'title': '나라사랑카드 분실ㆍ도난 시 어떻게 하나요?', 'score': 0.0}, {'id': 'narasarang_faq_006', 'title': '나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요?', 'score': 0.0}]\n[Guide LLM] model=/workspace/models/kakaocorp.kanana-nano-2.1b-instruct.Q4_K_M.gguf url=http://213.192.